#ML_Project

##Loading dataset

In [8]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Importing Libraries

In [31]:
import numpy as np
from scipy.stats import iqr,skew,kurtosis
from sklearn.svm import SVC
import math
from sklearn.metrics import accuracy_score, f1_score

## Loading the data

In [32]:
path = "/content/drive/MyDrive/ML_Project/dataset/"

### Loading the test data

In [33]:
test_labels = np.load(path +'/testing/testLabels.npy')
test_MS_Gyroscope = np.load(path +'/testing/testMSGyroscope.npy')
test_MS_Accelerometer = np.load(path +'/testing/testMSAccelerometer.npy')
test_Magnetometer = np.load(path + '/testing/testMagnetometer.npy')
test_linear_Acceleration = np.load(path + '/testing/testLinearAcceleration.npy')
test_jins_Gyroscope = np.load(path + '/testing/testJinsGyroscope.npy')
test_jins_Accelerometer = np.load(path + '/testing/testJinsAccelerometer.npy')
test_Gyroscope = np.load(path + '/testing/testGyroscope.npy')
test_Gravity = np.load(path + '/testing/testGravity.npy')
test_Accelerometer = np.load(path + '/testing/testAccelerometer.npy')

print(np.shape(test_labels))
print(np.shape(test_jins_Gyroscope))
print(np.shape(test_jins_Accelerometer))
print(np.shape(test_MS_Gyroscope))
print(np.shape(test_MS_Accelerometer))
print(np.shape(test_Magnetometer))
print(np.shape(test_Gyroscope))
print(np.shape(test_Gravity))
print(np.shape(test_Accelerometer))
print(np.shape(test_linear_Acceleration))

test_jins_Gyroscope=test_jins_Gyroscope.astype(np.float64)
test_jins_Accelerometer=test_jins_Accelerometer.astype(np.float64)
test_MS_Gyroscope=test_MS_Gyroscope.astype(np.float64)
test_MS_Accelerometer=test_MS_Accelerometer.astype(np.float64)
test_Magnetometer=test_Magnetometer.astype(np.float64)
test_Gyroscope=test_Gyroscope.astype(np.float64)
test_Gravity=test_Gravity.astype(np.float64)
test_Accelerometer=test_Accelerometer.astype(np.float64)
test_linear_Acceleration=test_linear_Acceleration.astype(np.float64)

test_data = []
test_data.append(test_jins_Gyroscope)
test_data.append(test_jins_Accelerometer)
test_data.append(test_MS_Gyroscope)
test_data.append(test_MS_Accelerometer)
test_data.append(test_Magnetometer)
test_data.append(test_Gyroscope)
test_data.append(test_Gravity)
test_data.append(test_Accelerometer)
test_data.append(test_linear_Acceleration)

(2288,)
(2288, 80, 3)
(2288, 80, 3)
(2288, 268, 3)
(2288, 268, 3)
(2288, 200, 3)
(2288, 800, 3)
(2288, 800, 3)
(2288, 800, 3)
(2288, 800, 3)


### Imporing training data

In [34]:
train_labels = np.load(path +'/training/trainLabels.npy')
train_MS_Gyroscope = np.load(path +'/training/trainMSGyroscope.npy')
train_MS_Accelerometer = np.load(path +'/training/trainMSAccelerometer.npy')
train_Magnetometer = np.load(path + '/training/trainMagnetometer.npy')
train_linear_Acceleration = np.load(path + '/training/trainLinearAcceleration.npy')
train_jins_Gyroscope = np.load(path + '/training/trainJinsGyroscope.npy')
train_jins_Accelerometer = np.load(path + '/training/trainJinsAccelerometer.npy')
train_Gyroscope = np.load(path + '/training/trainGyroscope.npy')
train_Gravity = np.load(path + '/training/trainGravity.npy')
train_Accelerometer = np.load(path + '/training/trainAccelerometer.npy')

print(np.shape(train_labels))
print(np.shape(train_jins_Gyroscope))
print(np.shape(train_jins_Accelerometer))
print(np.shape(train_MS_Gyroscope))
print(np.shape(train_MS_Accelerometer))
print(np.shape(train_Magnetometer))
print(np.shape(train_Gyroscope))
print(np.shape(train_Gravity))
print(np.shape(train_Accelerometer))
print(np.shape(train_linear_Acceleration))

train_jins_Gyroscope=train_jins_Gyroscope.astype(np.float64)
train_jins_Accelerometer=train_jins_Accelerometer.astype(np.float64)
train_MS_Gyroscope=train_MS_Gyroscope.astype(np.float64)
train_MS_Accelerometer=train_MS_Accelerometer.astype(np.float64)
train_Magnetometer=train_Magnetometer.astype(np.float64)
train_Gyroscope=train_Gyroscope.astype(np.float64)
train_Gravity=train_Gravity.astype(np.float64)
train_Accelerometer=train_Accelerometer.astype(np.float64)
train_linear_Acceleration=train_linear_Acceleration.astype(np.float64)

train_data = []
train_data.append(train_jins_Gyroscope)
train_data.append(train_jins_Accelerometer)
train_data.append(train_MS_Gyroscope)
train_data.append(train_MS_Accelerometer)
train_data.append(train_Magnetometer)
train_data.append(train_Gyroscope)
train_data.append(train_Gravity)
train_data.append(train_Accelerometer)
train_data.append(train_linear_Acceleration)


(2284,)
(2284, 80, 3)
(2284, 80, 3)
(2284, 268, 3)
(2284, 268, 3)
(2284, 200, 3)
(2284, 800, 3)
(2284, 800, 3)
(2284, 800, 3)
(2284, 800, 3)


## Feature Calculation Function

In [35]:
def calculate_features(data):
    features = []

    max = np.max(data, axis=1)
    for i in range(3):
      features.append(max[i])

    min = np.min(data, axis=1)
    for i in range(3):
      features.append(min[i])

    avg = []
    for i in range(3):
      a = np.average(np.abs(data[i] - np.median(data[i])))
      avg.append(a)
      features.append(a)

    std = np.std(data, axis=1)
    for i in range(3):
      features.append(std[i])

    for i in range(3):
      zero_crossing = len(np.where(np.diff(np.signbit(data[i]))))
      features.append(zero_crossing)

    per1 = np.percentile(data, 20, axis=1)
    for i in range(3):
      features.append(per1[i])

    per2 = np.percentile(data, 50, axis=1)
    for i in range(3):
      features.append(per2[i])

    per3 = np.percentile(data, 80, axis=1)
    for i in range(3):
      features.append(per3[i])

    for i in range(3):
      features.append(iqr(data[i]))

    for i in range(3):
      s = skew(data[i])
      features.append(s)

    for i in range(3):
      k = kurtosis(data[i])
      features.append(k)

    for i in range(3):
      features.append(np.correlate(data[i], data[i], mode='valid')[0])

    f_mean = []
    for i in range(3):
      first_order_mean = np.mean(data[i])
      f_mean.append(first_order_mean)
      features.append(first_order_mean)

    for i in range(3):
      norm_first_order_mean = np.linalg.norm(f_mean[i])
      features.append(norm_first_order_mean)

    s_mean = []
    for k in range(3):
      second_order_mean = np.var(data[k])
      s_mean.append(second_order_mean)
      features.append(second_order_mean)

    for k in range(3):
      norm_second_order_mean = np.linalg.norm(s_mean[k])
      features.append(norm_second_order_mean)

    for i in range(3):
      spectral_energy = sum(np.abs(np.fft.rfft(data[i]))**2)
      features.append(spectral_energy)

    for i in range(3):
      spectral_entropy = -sum(p * np.log2(p) for p in np.abs(np.fft.rfft(data[i]))**2 if p > 1e-12)
      features.append(spectral_entropy)

    return features


## Applying feature calc function on training data

In [36]:
train_feature_matrix = []
matrix = []
for i, data in enumerate(train_data):
    for j, mat in enumerate(data):
        matrix.append(calculate_features([mat[:,0], mat[:,1], mat[:,2]]))
    train_feature_matrix.append(matrix)
    matrix = []


<ipython-input-35-ef65c6bc8034>:42: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  s = skew(data[i])
<ipython-input-35-ef65c6bc8034>:46: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  k = kurtosis(data[i])


In [37]:
train_data=np.hstack([train_feature_matrix[0],train_feature_matrix[1],train_feature_matrix[2],train_feature_matrix[3],
           train_feature_matrix[4],train_feature_matrix[5],train_feature_matrix[6],train_feature_matrix[7],
           train_feature_matrix[8]])

In [38]:
np.array(train_feature_matrix).shape

(9, 2284, 54)

In [39]:
train_data.shape

(2284, 486)

## Dealing with NaN values

In [40]:
np.isnan(train_data).sum()

372

In [41]:
train_data=np.nan_to_num(train_data)

In [42]:
np.isnan(train_data).sum()

0

## Applying feature Calc function on testing data

In [43]:
test_feature_matrix = []
matrix = []
for i, data in enumerate(test_data):
    for j, mat in enumerate(data):
        matrix.append(calculate_features([mat[:,0], mat[:,1], mat[:,2]]))
    test_feature_matrix.append(matrix)
    matrix = []

<ipython-input-35-ef65c6bc8034>:42: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  s = skew(data[i])
<ipython-input-35-ef65c6bc8034>:46: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  k = kurtosis(data[i])


In [44]:
test_data=np.hstack([test_feature_matrix[0],test_feature_matrix[1],test_feature_matrix[2],test_feature_matrix[3],
           test_feature_matrix[4],test_feature_matrix[5],test_feature_matrix[6],test_feature_matrix[7],
           test_feature_matrix[8]])

test_data=np.nan_to_num(test_data)

## Creating a pipeline

In [45]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

def create_pipeline(model):
  return Pipeline([
      ('scaler',StandardScaler()),
      ('model',model)])

In [46]:
from sklearn.metrics import classification_report,accuracy_score,f1_score,recall_score
def get_metrics(model,xtest,ytrue):
  ypreds=np.array(model.predict(xtest))
  print(classification_report(ytrue,ypreds))
  return {'accuracy':accuracy_score(ytrue,ypreds),
          'recall':recall_score(ytrue,ypreds,average='macro'),
          'f1-score':f1_score(ytrue,ypreds,average='macro')}

## SVC

In [47]:
svc = create_pipeline(SVC(C=100))
svc.fit(train_data, train_labels)

Pipeline(steps=[('scaler', StandardScaler()), ('model', SVC(C=100))])

In [49]:
results=get_metrics(svc,test_data,test_labels)


              precision    recall  f1-score   support

           0       0.85      1.00      0.92        39
           1       0.82      0.94      0.87        52
           2       0.76      0.58      0.66        45
           3       0.63      0.80      0.70        40
           4       0.50      0.72      0.59        46
           5       0.54      0.67      0.60        42
           6       0.47      0.60      0.53        42
           7       0.29      0.34      0.31        41
           8       0.16      0.17      0.16        42
           9       0.60      0.60      0.60        20
          10       0.72      0.60      0.66        48
          11       0.68      0.69      0.68        36
          12       1.00      0.97      0.99        40
          13       0.53      0.57      0.55        42
          14       0.89      0.95      0.92        42
          15       0.93      0.96      0.95        45
          16       0.76      0.88      0.81        40
          17       0.50    

In [50]:
results

{'accuracy': 0.6774475524475524,
 'recall': 0.6777564273826368,
 'f1-score': 0.6744904068098813}

## Random Forest Classifier

In [55]:
from sklearn.ensemble import RandomForestClassifier
rfc = create_pipeline(RandomForestClassifier())
rfc.fit(train_data,train_labels)

Pipeline(steps=[('scaler', StandardScaler()),
                ('model', RandomForestClassifier())])

In [56]:
results=get_metrics(rfc,test_data,test_labels)

              precision    recall  f1-score   support

           0       0.91      1.00      0.95        39
           1       0.70      0.98      0.82        52
           2       0.74      0.38      0.50        45
           3       0.55      0.68      0.61        40
           4       0.44      0.76      0.56        46
           5       0.65      0.79      0.71        42
           6       0.48      0.33      0.39        42
           7       0.51      0.54      0.52        41
           8       0.18      0.17      0.17        42
           9       0.76      0.65      0.70        20
          10       0.73      0.67      0.70        48
          11       0.54      0.42      0.47        36
          12       1.00      0.95      0.97        40
          13       0.58      0.76      0.66        42
          14       0.85      0.93      0.89        42
          15       0.96      0.96      0.96        45
          16       0.76      0.93      0.83        40
          17       0.95    

In [57]:
results

{'accuracy': 0.6722027972027972,
 'recall': 0.6724781940859956,
 'f1-score': 0.665925634748932}